# Preprocesamiento de texto

Primero, importamos librerías.

In [1]:
import re, random
from typing import List
from collections import Counter
from tqdm.notebook import tqdm
import nltk
from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split

# Descargar recursos necesarios de NLTK
nltk.download('punkt')  # Tokenizador

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Preprocesamiento de texto

In [2]:
def preprocess_text(file_path: str) -> List[str]:
    """
    Procesa el texto de la ruta especificada, aplicando lo siguiente:

    - Tokenización por oración
    - Normalización
    - Reemplaza números por NUM
    - Añade tokens de inicio y finalización de oración <s> y </s>
    - Tokens con frecuencia unitaria se marcan como <UNK>

    Args:
        path (str): Ruta en la que leer el archivo de texto.

    Returns:
        List[str]: Listado de oraciones preprocesadas.
    """
    # Leer el archivo de texto línea por línea
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences = []
    for line in lines:
        # Dividir cada línea en oraciones
        line_sentences = sent_tokenize(line.strip())
        sentences.extend(line_sentences)
    
    # Listas donde guardar tokens y oraciones procesadas
    all_tokens = []
    processed_sentences = []
    
    # Para cada oración...
    for sentence in tqdm(sentences, desc="Procesando oraciones..."):

        # Pasar a minúsculas
        sentence = sentence.lower()
        
        # Separar números de letras (e.g. 9am -> 9 am)
        sentence = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', sentence)
        sentence = re.sub(r'([a-zA-Z])(\d+)', r'\1 \2', sentence)
        
        # Reemplazar números por el token NUM
        sentence = re.sub(r'\d+', 'NUM', sentence)
        
        # Eliminar puntuación y caracteres especiales
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        
        # Tokenizar palabras con nltk
        tokens = word_tokenize(sentence)
        
        # Añadir tokens de inicio y finalización
        tokens = ['<s>'] + tokens + ['</s>']
        
        # Agregar a las listas predefinidas si no es una oración vacía
        if len(tokens) > 2:
            all_tokens.extend(tokens)
            processed_sentences.append(tokens)
    
    # Contar la frecuencia de los tokens
    token_freq = Counter(all_tokens)
    
    # Reemplazar tokens de frecuencia unitaria con <UNK>
    final_sentences = []
    for tokens in processed_sentences:
        final_tokens = [token if token_freq[token] > 1 else '<UNK>' for token in tokens]
        final_sentences.append(final_tokens)
    
    return final_sentences

In [3]:
final_sentences_news = preprocess_text('consolidated_news.txt')
with open('processed_news.txt', 'w', encoding='utf-8') as output_file:
    for sentence in final_sentences_news:
        output_file.write(' '.join(sentence) + '\n')

Procesando oraciones...:   0%|          | 0/242822 [00:00<?, ?it/s]

In [4]:
final_sentences_bac = preprocess_text('consolidated_bac.txt')
with open('processed_bac.txt', 'w', encoding='utf-8') as output_file:
    for sentence in final_sentences_bac:
        output_file.write(' '.join(sentence) + '\n')

Procesando oraciones...:   0%|          | 0/7144331 [00:00<?, ?it/s]

In [5]:
len(final_sentences_news), len(final_sentences_bac)

(240500, 7088707)

In [6]:
def generate_training_testing(final_sentences: List[str], filename: str) -> None:
    sents = final_sentences.copy()
    random.shuffle(sents)

    # Determinar el punto de separación para 80-20
    split_index = int(0.8 * len(sents))

    # Separar en conjuntos de entrenamiento y prueba
    training_sentences = sents[:split_index]
    testing_sentences = sents[split_index:]

    # Guardar las oraciones de entrenamiento en un archivo txt
    with open(f'{filename}_training.txt', 'w', encoding='utf-8') as training_file:
        for sentence in training_sentences:
            training_file.write(' '.join(sentence) + '\n')

    # Guardar las oraciones de prueba en un archivo txt
    with open(f'{filename}_testing.txt', 'w', encoding='utf-8') as testing_file:
        for sentence in testing_sentences:
            testing_file.write(' '.join(sentence) + '\n')

In [7]:
generate_training_testing(final_sentences_news, "20N")
generate_training_testing(final_sentences_bac, "BAC")